In [ ]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV_all_dates includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV_all_dates
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [ ]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
from random import randint
from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA

# from myUtils import drop_symbols_all_NaN, yf_symbols_close, chunked_list # NOQA
from myUtils import drop_symbols_all_NaN, yf_symbols_close_1, chunked_list # NOQA

from myUtils import yf_download_AdjOHLCV, yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output
# verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'symbols_mktCap_2b.csv'  # symbols text file
filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_adjOHLCVA_downloaded = 'df_adjOHLCVA_downloaded'  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in adjOHLCV

In [ ]:
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

In [ ]:
# # Stop if Yahoo has not updated OHLCV data
# index_symbol = "XOM"  
# df_XOM = yf.download(index_symbol)
# download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')

# # yesterday = str(date.today() - timedelta(days = 1))
# # if yesterday == download_last_date:
# print(f'Yahoo data date: {download_last_date}, today: {str(date.today())}, weekday: {now.weekday()}')
# now = datetime.now()  # get current date and time
# if str(date.today()) == download_last_date and (now.weekday() != 5 or now.weekday() != 6):
  
#   msg_stop = f'Yahoo has not updated OHLCV data, today: {str(date.today())}, Yahoo download last date: {download_last_date}'
#   raise SystemExit(msg_stop)

In [ ]:
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
if download_last_date == df_adjOHLCV_last_date:
  msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
  raise SystemExit(msg_stop )

In [ ]:
# read symbols in file to a list
symbols_in_file = read_symbols_file(filename_symbols)
symbols_chunks = chunked_list(symbols_in_file, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]

In [ ]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  print(f'symbols_download_err: {symbols_download_err}')
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

In [ ]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
l_symbols_err 


In [ ]:
df = pd.concat(df_list, axis=1)

In [ ]:
df

In [ ]:
df.drop(l_symbols_err, axis=1, level=0, inplace=True)

In [ ]:
# df.columns
l_symbols = list(set(df.columns.get_level_values(0)))
l_symbols.sort()
l_symbols

In [ ]:
#sort columns by names
df = df[l_symbols]
df

In [ ]:
pickle_dump(df, path_data_dump, filename_pickled_df_adjOHLCVA_downloaded)

In [ ]:
df

In [ ]:
# l_symbols = list(set(df.columns.get_level_values(0)))
# l_symbols.sort()
df_adj_list=[]
# adjust OHLC
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

In [ ]:
df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
col_names = pd.MultiIndex.from_product([l_symbols, cols])
df_adj.columns = col_names

In [ ]:
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)

In [ ]:
# verify df test_symbols' close against Yahoo
test_symbols = ['A', 'SHEL', 'YUM']
# test_symbols = ['A', 'YUM']
# test_symbols = ['AAPL']
now = datetime.now()  # get current date and time
# if now.hour >= 20:  # only run this test after 8 pm when Yahoo data should be updated
#  run this test after 8 pm or on the weekend when Yahoo data should be updated

# if (now.hour >= 20) or now.weekday() == 5 or now.weekday() == 6:
if (now.hour >= 0) or now.weekday() == 5 or now.weekday() == 6:
  for symbol in test_symbols:
    s = df_adj.iloc[-222]
    sDate = s.name.strftime('%Y-%m-%d')

    sClose = s[symbol].Close
    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']
    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_stop = f'{symbol}  %_dif_Close > .0001'
      raise SystemExit(msg_stop)
    if symbol == test_symbols[-1]:
      msg_done = f"No errors found.  df test_symbols' Close matched Yahoo symbols' Close "
      print(msg_done)
else:
  print(f"Did not verify df test_symbols' close against Yahoo.  It's not 8 PM yet. Yahoo may not have updated their data.")

In [ ]:
print('Yahoo Website using yf.Ticker, close seems to be an Adjusted Close')
df_sym.loc['2021-12-18':'2021-12-26']

In [ ]:
df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)  

In [ ]:
df

In [ ]:
_df = df.SHEL
print('My Yahoo Download OHLCVA')
_df.loc['2021-12-18':'2021-12-26']

In [ ]:
print('My auto_adjust of OHLCVA')
df_adj.YUM.loc['2021-12-18':'2021-12-26']

In [ ]:
print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)

In [ ]:
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV_downloaded)
df

In [ ]:
df.AAPL

In [ ]:
# sort df by symbol
df_adjOHLCV_all_dates = df.sort_index(axis=1,level=0,sort_remaining=False)

In [ ]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV_all_dates, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV_all_dates, verbose)
# print(f'symbols_OHLCV: {symbols_OHLCV}')
print(f'symbols with all NaN dropped from df_adjOHLCV_all_dates: {symbols_dropped}')

In [ ]:
df_adjOHLCV_all_dates.AAPL

In [ ]:
# # rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
# #   symbols with all NaN has an added AdjClose column and will cause errors  
# #  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# #  .remove_unused_levels() prevents ValueError
# #   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# # The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
# df_adjOHLCV_all_dates.columns = df_adjOHLCV_all_dates.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV_all_dates.columns = df_adjOHLCV_all_dates.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [ ]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV_all_dates.reindex(df_XOM.index, fill_value=myNaN)

In [ ]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

In [ ]:
_df = df_adjOHLCV.copy()

In [ ]:
symbols_OHLCV = list(set([i[0] for i in list(_df)]))

In [ ]:
_df_symbols_close = _df.xs('Close', level=1, axis=1)
_df_symbols_close

In [ ]:
# create df of symbols' Close, sort df by symbols, pickled df
df_symbols_close, dates_dropped, symbols_OHLCV, symbols_dropped = yf_symbols_close_1(
    path_dir,
    path_data_dump,
    filename_pickled_df_adjOHLCV,
    verbose=verbose,
)
# multi-index sort df by symbol
df_symbols_close = df_symbols_close.sort_index(axis=1,level=0,sort_remaining=False)
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [ ]:
# retrieve pickled files
# print(f"Full path to pickled df_adjOHLCV_all_dates:  {path_data_dump}{filename_pickled_df_adjOHLCV_all_dates}")
# # df_all_dates = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV_all_dates, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
df_adjOHLCV_downloaded = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

In [ ]:
df
# df_symbols_close

In [ ]:
df.columns

In [ ]:
df.AAPL

In [ ]:
_df1 = df_adjOHLCV_downloaded
_df1
_df_cl = _df1.xs('Close', level=1, axis=1)
_df_cl

In [ ]:
yf_aapl = yf.download('AAPL')

In [ ]:
yf_aapl

In [ ]:
df_adjOHLCV_downloaded.columns